In [ ]:
import xgboost as xgb
import pandas as pd
from sklearn.model_selection import train_test_split

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pulp import *
import pandas as pd
import os, glob
import seaborn as sns
from scipy.stats import kruskal
import scikit_posthocs as sp
from scipy.stats import mannwhitneyu

load_dotenv('./Credentials.env',override=True)

os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = str(os.getenv("GOOGLE_APPLICATION_CREDENTIALS"))  
os.environ['GCLOUD_PROJECT'] =  str(os.getenv("GCLOUD_PROJECT"))

%load_ext google.cloud.bigquery

from google.cloud import bigquery
client=bigquery.Client()
from google.cloud import bigquery_storage_v1

## Read patients_Notes_EHR

In [ ]:
%%bigquery Train_set_df
select * from `som-nero-phi-jonc101.blood_culture_stewardship.cohort` where order_year>=2015 and order_year<2022

In [ ]:
%%bigquery Val_set_df
select * from `som-nero-phi-jonc101.blood_culture_stewardship.cohort` where order_year>=2022 and order_year<2023

In [ ]:
%%bigquery Test_set_df
select * from `som-nero-phi-jonc101.blood_culture_stewardship.cohort` where order_year>=2023 

In [ ]:
%%bigquery Notes_df
select * from `som-nero-phi-jonc101.blood_culture_stewardship.EDProviderNotes`

In [ ]:
Train_set_df_note=pd.merge(Train_set_df,Notes_df,how='inner',on=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'])
Train_set_df_note[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']].drop_duplicates().shape

Val_set_df_note=pd.merge(Val_set_df,Notes_df,how='inner',on=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'])
Val_set_df_note[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']].drop_duplicates().shape

Test_set_df_note=pd.merge(Test_set_df,Notes_df,how='inner',on=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded'])
Test_set_df_note[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']].drop_duplicates().shape

In [ ]:
Train_set_df_note=Train_set_df_note[Train_set_df_note.deid_note_text.notnull()]
Train_set_df_note[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']].drop_duplicates().shape

Val_set_df_note=Val_set_df_note[Val_set_df_note.deid_note_text.notnull()]
Val_set_df_note[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']].drop_duplicates().shape

Test_set_df_note=Test_set_df_note[Test_set_df_note.deid_note_text.notnull()]
Test_set_df_note[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded']].drop_duplicates().shape

# Approach 1

In [ ]:
Train_set_df_note.drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','blood_culture_order_datetime','notedatetime','deid_note_text'],inplace=True)
Train_set_df_note.shape

Val_set_df_note.drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','blood_culture_order_datetime','notedatetime','deid_note_text'],inplace=True)
Val_set_df_note.shape

Test_set_df_note.drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','blood_culture_order_datetime','notedatetime','deid_note_text'],inplace=True)
Test_set_df_note.shape


In [ ]:
Train_set_df_note['blood_culture_order_datetime'] = pd.to_datetime(Train_set_df_note['blood_culture_order_datetime'])
Train_set_df_note['notedatetime'] = pd.to_datetime(Train_set_df_note['notedatetime'])
Train_set_df_note['time-diff']=(Train_set_df_note['blood_culture_order_datetime']-Train_set_df_note['notedatetime']).dt.total_seconds()/3600
Train_set_df_note['time-diff']=Train_set_df_note['time-diff'].astype(int)
Train_set_df_note=Train_set_df_note[Train_set_df_note['time-diff']>=-24]


Val_set_df_note['blood_culture_order_datetime'] = pd.to_datetime(Val_set_df_note['blood_culture_order_datetime'])
Val_set_df_note['notedatetime'] = pd.to_datetime(Val_set_df_note['notedatetime'])
Val_set_df_note['time-diff']=(Val_set_df_note['blood_culture_order_datetime']-Val_set_df_note['notedatetime']).dt.total_seconds()/3600
Val_set_df_note['time-diff']=Val_set_df_note['time-diff'].astype(int)
Val_set_df_note=Val_set_df_note[Val_set_df_note['time-diff']>=-24]

Test_set_df_note['blood_culture_order_datetime'] = pd.to_datetime(Test_set_df_note['blood_culture_order_datetime'])
Test_set_df_note['notedatetime'] = pd.to_datetime(Test_set_df_note['notedatetime'])
Test_set_df_note['time-diff']=(Test_set_df_note['blood_culture_order_datetime']-Test_set_df_note['notedatetime']).dt.total_seconds()/3600
Test_set_df_note['time-diff']=Test_set_df_note['time-diff'].astype(int)
Test_set_df_note=Test_set_df_note[Test_set_df_note['time-diff']>=-24]


In [ ]:
Train_set_df_note['time-diff']=(Train_set_df_note['blood_culture_order_datetime']-Train_set_df_note['notedatetime']).dt.total_seconds().abs()
Val_set_df_note['time-diff']=(Val_set_df_note['blood_culture_order_datetime']-Val_set_df_note['notedatetime']).dt.total_seconds().abs()
Test_set_df_note['time-diff']=(Test_set_df_note['blood_culture_order_datetime']-Test_set_df_note['notedatetime']).dt.total_seconds().abs()


Train_set_df_note = Train_set_df_note.sort_values('time-diff').drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded', 'blood_culture_order_datetime'], keep='first')
Val_set_df_note = Val_set_df_note.sort_values('time-diff').drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded', 'blood_culture_order_datetime'], keep='first')
Test_set_df_note = Test_set_df_note.sort_values('time-diff').drop_duplicates(subset=['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded', 'blood_culture_order_datetime'], keep='first')


In [ ]:
Train_set_df_note['doc_id'] = Train_set_df_note.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','blood_culture_order_datetime']).ngroup() + 1
Val_set_df_note['doc_id'] = Val_set_df_note.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','blood_culture_order_datetime']).ngroup() + 1
Test_set_df_note['doc_id'] = Test_set_df_note.groupby(['anon_id', 'pat_enc_csn_id_coded', 'order_proc_id_coded','blood_culture_order_datetime']).ngroup() + 1

In [ ]:
## Evalute GPT4 on whole Notes 

## Evalute GPT4 on whole Notes + EHR ********* (running from this note book)

In [ ]:
Test_set_df_note['Label']=(Test_set_df_note['positive_blood_culture']| Test_set_df_note['positive_blood_culture_in_week'])

In [ ]:
EHR=[col for col in Test_set_df_note.columns.values if 'min' in col]
EHR.extend([col for col in Test_set_df_note.columns.values if 'max' in col])
EHR.extend([col for col in Test_set_df_note.columns.values if 'avg' in col])
EHR.extend([col for col in Test_set_df_note.columns.values if 'median' in col])
EHR

In [ ]:
def format_structured_features(row, ehr_columns):
    return "\n".join([f"- {col}: {row[col]}" for col in ehr_columns if pd.notnull(row[col])])


def dict_structured_features(row, ehr_columns):
    EHR={}
    for col in ehr_columns:
        if pd.notnull(row[col]):
            EHR[col]=row[col]
    return EHR
    

prompt_records = []

def split_note_text(note_text, max_words=1000, overlap=10):
    words = note_text.split()
    chunks = []
    i = 0
    while i < len(words):
        chunk = words[i:i+max_words]
        chunks.append(" ".join(chunk))
        i += max_words - overlap
    return chunks


EHR_IncludeInprompt=True

for i, row in Test_set_df_note.iterrows():
    structured_block = format_structured_features(row, EHR)
    structured_dict = dict_structured_features(row, EHR)
    note_text = row['deid_note_text']
    note_text = re.sub(r'\s+', ' ', note_text.replace('\n', ' ')).strip()
    
    # Split if longer than 3000 words
    if len(note_text.split()) > 1000:
        note_chunks = split_note_text(note_text)
    else:
        note_chunks = [note_text]

    # Generate one prompt per chunk
    for chunk_index, chunk in enumerate(note_chunks):
        if EHR_IncludeInprompt:
            prompt = (
                "You are an infectious disease expert. Based on the patient information below, determine whether this patient's blood culture is likely to be positive.\n"
                "Respond with **only** 'Yes' or 'No'. Reply 'Yes' if a positive blood culture is likely; otherwise, reply 'No'.\n\n"
                "Patient Information:\n"
                f"{structured_block}\n\n"
                "Clinical Note:\n"
                f"{chunk}\n"
            )
        else:
            prompt = (
                "You are an infectious disease expert. Based on the patient information below, determine whether this patient's blood culture is likely to be positive.\n"
                "Respond with **only** 'Yes' or 'No'. Reply 'Yes' if a positive blood culture is likely; otherwise, reply 'No'.\n\n"
                "Patient Information:\n"
                "Clinical Note:\n"
                f"{chunk}\n"
            )
            
        prompt_records.append({
            'anon_id': row['anon_id'],
            'pat_enc_csn_id_coded': row['pat_enc_csn_id_coded'],
            'order_proc_id_coded': row['order_proc_id_coded'],
            'blood_culture_order_datetime':row['blood_culture_order_datetime'],
            'Label':row['Label'],
            'doc_id':row['doc_id'],
            'chunk_index': chunk_index,
            'prompt': prompt,
            'EHR':[structured_dict]
        })

# Convert to DataFrame

Test_set_prompt_df = pd.DataFrame(prompt_records)

In [ ]:
Test_set_prompt_df

In [ ]:
import requests
import json
my_key = "xxxxxxxxxx"
headers = {'Ocp-Apim-Subscription-Key': my_key, 'Content-Type': 'application/json'}

url = "https://apim.stanfordhealthcare.org/openai-eastus2/deployments/gpt-4.1/chat/completions?api-version=2025-01-01-preview" 


In [ ]:
import pdb
import json
import requests
import pdb
from time import sleep

def gpt4response(row, model="gpt-4",EHR_IncludeInprompt=True):
    prompt=row['prompt']
    ehr_data = row["EHR"]
    doc_id=row['doc_id']
    ehr_data=row['EHR']
    if not prompt:
        return None

    if EHR_IncludeInprompt:
        payload = json.dumps({
            "model": model,
            "messages": [{"role": "user", "content": prompt}
        ]

        })
    else:
        payload = json.dumps({
        "model": model,
        "messages": [
            {"role": "user", "content": prompt},
            {"role": "system", "content": f"Here is the structured EHR data in JSON format:\n{json.dumps(ehr_data)}"}
        ]
    })

    while True:
        try:
            response = requests.post(url, headers=headers, data=payload)
            response_json = response.json()
            if 'choices' in response_json:
                return response_json['choices'][0]['message']['content']
            else:
                print(f"doc_id:{doc_id}")
                print("Response missing 'choices'. Retrying in 10 seconds...")
                sleep(10)
        except Exception as e:
            print(f"Error: {e}. Retrying in 10 seconds...")
            sleep(10)

In [ ]:
Test_set_prompt_df.sort_values(by=['doc_id'],inplace=True)
Test_set_prompt_df = Test_set_prompt_df.reset_index(drop=True)
Test_set_prompt_df

In [ ]:
len_batches=100
n_batches=int(Test_set_prompt_df.shape[0]/len_batches)
for i in range(n_batches):
    DF=Test_set_prompt_df.iloc[i*len_batches:i*len_batches+len_batches]
    DF['GPT_response']=DF.apply(lambda row:gpt4response(row),axis=1)
    filename=f"GPT4_Testset_WholeNotes_EHR_part{i}.csv"
    DF.to_csv(filename,index=False)
    print(f'Batch{i} is done')

print(i)

In [ ]:
DF=Test_set_prompt_df.iloc[i*len_batches:]
DF['GPT_response']=DF.apply(lambda row:gpt4response(row),axis=1)
filename=f"GPT4_Testset_WholeNotes_EHR_part{i}.csv"
DF.to_csv(filename,index=False)
print(f'Batch{i} is done')

### Evalute_GPT4 responses

In [ ]:
import glob
Res=[]
for file in glob.glob('./*.csv'):
    if 'GPT4_Testset_WholeNotes_EHR_part' in file:
        if not len(Res):
            Res=pd.read_csv(file)
        else:
            Res=pd.concat([Res,pd.read_csv(file)],axis=0)
Res.head()
        
        

In [ ]:
Res.loc[Res.GPT_response=='No','GPT_response']=0
Res.loc[Res.GPT_response=='No.','GPT_response']=0
Res.loc[Res.GPT_response=='Yes','GPT_response']=1

Res=Res[['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','blood_culture_order_datetime','Label','GPT_response']]
Res.drop_duplicates(inplace=True)
test=Res.groupby(by=['anon_id','pat_enc_csn_id_coded','order_proc_id_coded','blood_culture_order_datetime','Label'],as_index=False).max()
test.drop_duplicates(inplace=True)
test
test['Label']=test['Label'].astype('int')

In [ ]:
from sklearn.metrics import confusion_matrix, f1_score
import pandas as pd

test['Label'] = test['Label'].astype(int)
test['GPT_response'] = test['GPT_response'].astype(int)

y_true = test['Label']
y_pred =test['GPT_response']

# Confusion matrix: [[TN, FP], [FN, TP]]
tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

# Metrics
sensitivity = tp / (tp + fn) if (tp + fn) > 0 else 0
specificity = tn / (tn + fp) if (tn + fp) > 0 else 0
ppv         = tp / (tp + fp) if (tp + fp) > 0 else 0
npv         = tn / (tn + fn) if (tn + fn) > 0 else 0
f1          = f1_score(y_true, y_pred)

# Print results
print(f"Sensitivity (Recall): {sensitivity:.2f}")
print(f"Specificity: {specificity:.2f}")
print(f"PPV (Precision): {ppv:.2f}")
print(f"NPV: {npv:.2f}")
print(f"F1 Score: {f1:.2f}")